In [1]:
GPT_CONFIG = {
    "vocab_size": 50257,
    "context_length": 1024,
    "embedding_dimensions": 768,
    "num_heads": 12,
    "num_layers": 12,
    "dropout_rate": 0.1,
    "qkv_bias": False,
}

In [8]:
#placeholder architecture
import torch
import torch.nn as nn

class DummyGPTModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embeddings = nn.Embedding(config["vocab_size"], config["embedding_dimensions"])
        self.position_embeddings = nn.Embedding(config["context_length"], config["embedding_dimensions"])
        self.dropout = nn.Dropout(config["dropout_rate"])
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(config) for _ in range(config["num_layers"])]
        )
        self.final_norm = DummyLayerNorm(config["embedding_dimensions"])
        self.out_head = nn.Linear(
            config["embedding_dimensions"], config["vocab_size"], bias=False
            )

    def forward(self, in_idx):
        batch_size, seq_length = in_idx.shape
        token_embeddings = self.token_embeddings(in_idx)
        position_embeddings = self.position_embeddings(
            torch.arange(seq_length, device=in_idx.device)
        )
        x = token_embeddings + position_embeddings
        x = self.dropout(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits
    
class DummyTransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()

    def forward(self, x):
            return x
        
class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()

    def forward(self, x):
            return x

In [9]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
batch = []
text1 = "Every effort moves you"
text2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(text1)))
batch.append(torch.tensor(tokenizer.encode(text2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [10]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG)
logits = model(batch)
print(logits.shape)
print(logits)

torch.Size([2, 4, 50257])
tensor([[[-1.2034,  0.3201, -0.7130,  ..., -1.5548, -0.2390, -0.4667],
         [-0.1192,  0.4539, -0.4432,  ...,  0.2392,  1.3469,  1.2430],
         [ 0.5307,  1.6720, -0.4695,  ...,  1.1966,  0.0111,  0.5835],
         [ 0.0139,  1.6754, -0.3388,  ...,  1.1586, -0.0435, -1.0400]],

        [[-1.0908,  0.1798, -0.9484,  ..., -1.6047,  0.2439, -0.4530],
         [-0.7860,  0.5581, -0.0610,  ...,  0.4835, -0.0077,  1.6621],
         [ 0.3567,  1.2698, -0.6398,  ..., -0.0162, -0.1296,  0.3717],
         [-0.2407, -0.7349, -0.5102,  ...,  2.0057, -0.3694,  0.1814]]],
       grad_fn=<UnsafeViewBackward0>)
